In [1]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append("../")

In [2]:
import re
import stanza
from collections import Counter
stanza.download('en')  

2020-04-29 00:17:40 INFO: Downloading default packages for language: en (English)...
2020-04-29 00:17:41 INFO: File exists: /home/tdimson/stanza_resources/en/default.zip.
2020-04-29 00:17:44 INFO: Finished downloading models and saved to /home/tdimson/stanza_resources.


In [3]:
import itertools
import datasets
import pickle
from transformers import AutoModelWithLMHead, AutoTokenizer

In [4]:

def print_words(words, f):
    for word in words:
        word_str = [word.word]
        if word.pos:
            word_str.append(f"/{word.pos}/")
        if word.topic:
            word_str.append(f"[{word.topic}]")
        print(" ".join(word_str), file=f)
        print(f"\t{word.definition}", file=f)
        print(f"\t\"{word.example}\"{' |e|' if word.from_example_expansion else ''}", file=f)

        print("", file=f)

2020-04-28 23:37:24 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-04-28 23:37:24 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |

2020-04-28 23:37:24 INFO: Use device: gpu
2020-04-28 23:37:24 INFO: Loading: tokenize
2020-04-28 23:37:24 INFO: Loading: pos
2020-04-28 23:37:25 INFO: Done loading processors!


Counter({'VBD': 3, 'NNP': 0, 'VBN': 0, 'IN': 0, 'POS': 0, 'NN': 0, '.': 0, 'PRP': 0, 'CD': 0, 'CC': 0, 'VBZ': 0})


'VBD'

In [5]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,mwt,pos')
tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.add_special_tokens(datasets.SpecialTokens.special_tokens_dict())
blacklist = set((x.lower() for x in itertools.chain.from_iterable(
    [e.word] + e.derivatives
    for e in pickle.load(open(f"../data/en_dictionary_parsed_randomized.pickle", "rb")))
))
model = AutoModelWithLMHead.from_pretrained("../models/en_dictionary_parsed_lr_00001/checkpoint-120000").to("cuda:0")

2020-04-29 00:17:58 WARNING: Can not find mwt: default from official model list. Ignoring it.
2020-04-29 00:17:58 INFO: Loading these models for language: en (English):
| Processor | Package |
-----------------------
| tokenize  | ewt     |
| pos       | ewt     |

2020-04-29 00:17:58 INFO: Use device: gpu
2020-04-29 00:17:58 INFO: Loading: tokenize
2020-04-29 00:17:59 INFO: Loading: pos
2020-04-29 00:18:00 INFO: Done loading processors!


In [15]:
prefix

'<|bod|>dasfdagsmn24134fd.s~!'

In [44]:
words, stats = datasets.ParsedDictionaryDefinitionDataset.generate_words(
    tokenizer, model,
    num=10,
    max_iterations=1, 
    blacklist=blacklist, 
    do_example_expansion=True, 
    generation_args=dict(
        top_k=300,
        num_return_sequences=50,
        max_length=256,
        do_sample=True,
    ),
    expansion_generation_overrides=dict(
        top_k=50,
        num_return_sequences=20,
        do_sample=True,
    ),
    num_expansion_candidates=20,
    example_match_pos_pipeline=nlp,
)

print(stats)
print()
print_words(words, sys.stdout)

iterations=1 | items_considered 1.00@50, failed_match 0.00@0, blacklist_filtered 0.68@34, seen_filtered 0.00@0, proper_noun_filtered 0.08@4, example_filtered 0.00@0, example_expansions 0.20@10, example_expansion_success 0.14@7, returned 0.18@9

dipeterallopyelia /noun/
	dimple dyes that color the eye with dewy- <|bd|> dipeterallopyelia esters
	"the dipeterallopyelia coloration" |e|

reentry /noun/
	an admission or increase to one's financial liabilities arising from the reentry of property or goods
	"a reentry of the board of directors"

lappurine /noun/ [historical]
	a small vessel with a narrow bottom and one or more sail pylons.
	"a ferry that carried lappurine across the Thames" |e|

heptathione nephinium /noun/
	a bacterium related to heptaparum, closely related to E. coli, which causes yeast infection and can also cause dengue and other fever.
	"a bacterium from Asia is the most common heptathione nephinium." |e|

cobold /noun/
	a silver-crusted goldfish that lives in coastal wat